In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/XiaoXiang_project')

Mounted at /content/gdrive


In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import re
import jieba
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from lightgbm import LGBMClassifier

train_df = pd.read_csv('./data/train_chinese.csv')
train_df = train_df[~(train_df['Age']==0)]
train_df = train_df[~(train_df['Gender']==0)]
train_df = train_df[~(train_df['Education']==0)]

In [27]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), max_df=0.95, min_df=2, max_features=10000).fit(train_df['Query_List'].iloc[:].values)
train_tfidf = tfidf.transform(train_df['Query_List'].iloc[:].values)

from scipy.sparse import hstack
train = hstack((train_tfidf,train_df['Education'].iloc[:].values.reshape(-1, 1)), format='csr')

skf = StratifiedKFold(n_splits=5, random_state=42)

for idx, (train_index, valid_index) in enumerate(skf.split(train, train_df['Age'].values)):
  x_train_, x_valid_ = train[train_index], train[valid_index]
  y_train_, y_valid_ = train_df['Age'].values[train_index], train_df['Age'].values[valid_index]
  
  clf = LGBMClassifier()
  # clf = SVC()

  clf.fit(x_train_, y_train_)
  val_pred = clf.predict(x_valid_)
  
  print(f1_score(y_valid_, val_pred, average='macro'))

0.4319974474776875
0.4395835904571102
0.4370017365107906
0.4432637624781856
0.4411691179383668
